In [1]:
import numpy as np
import pandas as pd
import csv
import os
#from SepsisCheck import sepsischeck_utilities_for_pkl as su
from sklearn.metrics import precision_recall_fscore_support as score
#from sklearn.metrics import average_precision_score as score
from sklearn.metrics import classification_report as report
from sklearn.metrics import roc_auc_score as auroc
from sklearn.metrics import confusion_matrix

In [2]:
lab = pd.read_csv("patients_no_text_sepsis_labels.csv")
forecast = pd.read_pickle("4OBS12forecasting_preds_test.pkl")
hours = pd.read_csv("lab_hours.csv")
lab["hour"] = hours["hour"]

In [4]:
IDs = pd.read_csv("IDs.csv")
train_IDs = np.unique(IDs["train_ids"][~np.isnan(IDs["train_ids"])])
test_IDs = np.unique(IDs["test_ids"][~np.isnan(IDs["test_ids"])])
val_IDs = np.unique(IDs["val_ids"][~np.isnan(IDs["val_ids"])])
print(np.intersect1d(train_IDs,test_IDs))
print(np.intersect1d(train_IDs,val_IDs))
print(np.intersect1d(test_IDs,val_IDs))

[]
[]
[]


In [5]:
temp_ = lab.copy()
temp_["granular_label"] = np.where(((temp_["m:charttime"] >= temp_["m:presumed_onset"]) & (temp_["m:presumed_onset"] > 0.0)), 1, 0)
rows = temp_.drop(columns=["sepsis_label", "m:charttime", "traj", "r:reward", "a:action", "o:output_4hourly", "o:output_total", "o:PaO2_FiO2", "Unnamed: 0", "Unnamed: 0.1", "m:presumed_onset", "o:SOFA", "o:SIRS", "o:input_4hourly", "o:input_total", "o:GCS", "o:PTT", "o:DiaBP", "o:MeanBP", "o:SpO2", "o:cumulated_balance", "o:Creatinine", "o:Platelets_count", "o:re_admission", "o:mechvent", "o:max_dose_vaso"])
r = rows.copy()
r = r.loc[r["step"]<=4]
r.drop(columns="step", inplace=True)
X_train = r.loc[r["m:icustayid"].isin(train_IDs)]
y_train = X_train["granular_label"]
X_test = r.loc[r["m:icustayid"].isin(test_IDs)]
y_test = X_test["granular_label"]
X_val = r.loc[r["m:icustayid"].isin(val_IDs)]
y_val = X_val["granular_label"]


X_train.drop(columns=["granular_label", "m:icustayid"], inplace=True)
X_test.drop(columns=["granular_label", "m:icustayid"], inplace=True)
X_val.drop(columns=["granular_label", "m:icustayid"], inplace=True)

/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_55917/106022131.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["granular_label", "m:icustayid"], inplace=True)
/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_55917/106022131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns=["granular_label", "m:icustayid"], inplace=True)
/var/folders/rm/v29286yx225c2ktvpkpwfr5r0000gq/T/ipykernel_55917/106022131.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [6]:
strats_df = forecast[0]
var_map_hour = list(forecast[1].keys())
var_map_hour.append("hour")
var_map = list(forecast[1].keys())